In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import random
import os

if not os.path.exists('./restList'):
    os.mkdir('./restList')
    
if not os.path.exists('./restPhoto'):
    os.mkdir('./restPhoto')

url='https://www.tripadvisor.com.tw/Restaurants-g293913-Taipei.html'

userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36 Edg/90.0.818.56'
headers = {'User-Agent':userAgent}

browser=webdriver.Chrome()

for i in range(3):
    browser.get(url)
    res=browser.page_source
    soup=BeautifulSoup(res, 'html.parser')

    for restTags in soup.select('a[class="_15_ydu6b"]'):
        try:
            restUrl='https://www.tripadvisor.com.tw'+restTags['href']
            browser.get(restUrl)
            restRes=browser.page_source
            restSoup=BeautifulSoup(restRes, 'html.parser')
            restName=restSoup.select('h1[class="_3a1XQ88S"]')[0].text
            genRating=restSoup.select('svg[class="zWXXYhVR"]')[0]['title'].split(' of')[0]
            price=restSoup.select('a[class="_2mn01bsa"]')[0].text
            ranking=restSoup.select('a[class="_15QfMZ2L"]')[0].text
            location=restSoup.select('a[class="_15QfMZ2L"]')[1].text
            tel=restSoup.select('span[class="_15QfMZ2L"]')[0].text
            website=restSoup.select('a[class="_2wKz--mA _15QfMZ2L"]')[0]['href']
            imgUrl=restSoup.select('img[class="basicImg"]')[0]['data-lazyurl']
            resImg = requests.get(imgUrl, headers=headers)
            resContent = resImg.content
#             with open('./restPhoto/{}.{}'.format(restName, imgUrl.split('.')[-1]), 'wb') as f:
#                 f.write(resContent)
            with open('./restList/{}.txt'.format(restName), 'w', encoding='utf-8') as f:
                f.write('餐廳名稱: '+restName+'\n')
                f.write('官方網站: '+website+'\n')
                f.write('區域排名: '+ranking+'\n')
                f.write('餐廳地點: '+location+'\n')
                f.write('餐廳電話: '+tel+'\n')
                f.write('餐廳價位: '+price+'\n')
                f.write('整體評分: '+genRating+'\n')
        except IndexError:
            with open('./restList/{}.txt'.format(restName), 'w', encoding='utf-8') as f:
                f.write('餐廳名稱: '+restName+'\n')
            print(restName+'資料不全')

        with open('./restList/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
            f.write('細項評分: ')
        try:
            for ratings in restSoup.select('div[class="jT_QMHn2"]'):
                bubbles=ratings.select('span[class="_377onWB-"] span')[0]['class'][1].split('bubble_')[-1]
                with open('./restList/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
                    f.write(ratings.text+'- ')
                    f.write(str(int(bubbles)/10)+'/ ')
            with open('./restList/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
                f.write(' '+'\n')
        except IndexError:
            pass
                        
        with open('./restList/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
            f.write('餐廳類型: ')
        try:
            for restType in restSoup.select('a[class="_2mn01bsa"]')[1:]:
                with open('./restList/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
                    f.write(restType.text+'/ ')
            with open('./restList/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
                f.write(' '+'\n')
        except IndexError:
            pass
        
        time.sleep(random.randint(1, 5))
        
    newUrl = 'https://www.tripadvisor.com'+soup.select('a[class="nav next rndBtn ui_button primary taLnk"]')[0]['href']
    url=newUrl
    
browser.close()

原烧 (台北林森北店)資料不全


In [6]:
import pandas as pd
import glob

data = list()
for filePath in glob.glob('./restList/*.txt'):
    with open(filePath, 'r', encoding='utf-8') as f:
        tmpList = f.read().split('\n')
        data.append(tmpList)
columns = [r.split(': ') [0] for r in data[0]]

df = pd.DataFrame(data=data, columns=columns)

for c in df:
    df[c] = df[c].apply(lambda x: x.split(': ')[-1])
df

,餐廳名稱,官方網站,區域排名,餐廳地點,餐廳電話,餐廳價位,整體評分,細項評分,餐廳類型,
0,168牛排馆,http://www.grandvictoria.com.tw/dining/ins.php...,"中山區排名第 #9 的餐廳，共 1,653 名",台灣 中山區台北 中山区敬业四路168号,+886 2 6602 5678,$$$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.5/,牛排/ 歐式料理/ 提供無麩質選擇/,
1,A Cheng Goose,https://www.acheng.com.tw/,"中山區排名第 #15 的餐廳，共 1,653 名","台灣 中山區台北 No. 105, Jilin Rd., Zhongshan Dist",+886 2 2541 5238,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.0/ 實惠- 4.5/,中式料理/ 亞洲料理/ 台灣小吃/台菜/,
2,AISIN GIORO Restaurant,http://www.emperors.com.tw,"大安排名第 #6 的餐廳，共 2,036 名",106 台灣 大安台北 大安区市民大道四段104号1楼,+886 2 2778 6580,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.5/,中式料理/ 海鮮/ 燒烤/,
3,Aplus Dining Sake Bar,http://www.aplusdininggroup.com/index_e.php,"大安排名第 #19 的餐廳，共 2,036 名","106 台灣 大安台北 No. 33, Sec. 1, Anhe Rd., Daan Dist",+886 2 2731 9266,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.5/,日式料理/ 酒吧/ 壽司/,
4,"Bencotto at Mandarin Oriental, Taipei",https://www.mandarinoriental.com/taipei/songsh...,松山排名第 #7 的餐廳，共 922 名,10548 台灣 松山台北 敦化北路158号,+886 2 2715 6868,$$$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.5/,義式料理/ 提供素食選擇/ 提供純素選擇/,
...,...,...,...,...,...,...,...,...,...,...
84,鼎泰丰(101店),http://www.dintaifung.com.tw,信義區排名第 #11 的餐廳，共 936 名,110 台灣 信義區台北 市府路45号 B1,+886 2 8101 7799,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.0/,台灣小吃/台菜/ 提供素食選擇/ 提供純素選擇/,
85,鼎泰丰(信义新光A4店),http://www.dintaifung.com.tw/,信義區排名第 #10 的餐廳，共 936 名,110 台灣 信義區台北 信义区松高路19号 B2,+886 2 2345 2528,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/,中式料理/ 亞洲料理/ 台灣小吃/台菜/,
86,鼎泰丰(信义路),http://www.dintaifung.com.tw/eng/store.php?cid...,"中正區排名第 #3 的餐廳，共 1,032 名",106 台灣 中正區台北 大安区信义路二段194号 MRT Dongmen Station....,+886 2 2321 8928,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.0/,中式料理/ 亞洲料理/ 台灣小吃/台菜/,
87,鼎泰丰(复兴店),http://www.dintaifung.com.tw/tw/default.htm,"大安排名第 #15 的餐廳，共 2,036 名",10654 台灣 大安台北 忠孝东路三段300号B2,+886 2 8772 0528,$$ - $$$,4.5 分 (共 5 分),食物- 4.5/ 服務- 4.5/ 實惠- 4.0/ 氛圍- 4.0/,中式料理/ 亞洲料理/ 台灣小吃/台菜/,


In [3]:
df.to_csv('./restList.csv', index=False, encoding='utf-8-sig')